In this notebook, we will demonstrate the concept of ensemble learning. With ensemble models, we combine multiple models together to create a more accurate prediction.

In [99]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Load the dataset
iris = datasets.load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)





In [100]:
# Pipeline: Integrate preprocessing into the training process
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', DecisionTreeClassifier())
])

# Number of bootstrap samples and trees in the ensemble
n_samples = len(X_train)
n_trees = 50

def bootstrap_sample(X, y):
    indices = np.random.choice(len(X), size=len(X), replace=True)
    return X[indices], y[indices]



In [101]:
# Train multiple decision trees on different bootstrap samples
trees = []
for _ in range(n_trees):
    X_sample, y_sample = bootstrap_sample(X_train, y_train)
    
    # Clone the pipeline for each tree
    cloned_pipeline = Pipeline(steps=[('scaler', StandardScaler()), ('clf', DecisionTreeClassifier())])
    cloned_pipeline.fit(X_sample, y_sample)
    trees.append(cloned_pipeline)



In [102]:
trees

[Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassif

In [103]:
# Aggregate predictions
predictions = np.zeros((len(X_test), n_trees))
for tree_index, tree in enumerate(trees):
    predictions[:, tree_index] = tree.predict(X_test)



In [104]:
predictions

array([[1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [2., 2., 2., ..., 2., 2., 2.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [105]:
# Majority vote
y_pred = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=predictions.astype('int'))



In [106]:
# Test accuracy
print(f"Accuracy (Custom Bagged Trees): {accuracy_score(y_test, y_pred)*100:.2f}%")

Accuracy (Custom Bagged Trees): 100.00%


### Practice time with titanic

In [107]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import seaborn as sns

# Load the dataset
titanic = sns.load_dataset('titanic')
X = titanic.drop(columns=['survived'])  # Features
y = titanic['survived']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Now you can proceed with your scikit-learn code


In [108]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [109]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [120]:
titanic.describe()

,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [119]:
titanic = titanic.drop('deck',axis=1)

In [122]:
titanic['age'].fillna(30, inplace=True)

In [110]:
# Pipeline: Integrate preprocessing into the training process
#pipeline = Pipeline([
 
 #   ('scaler', StandardScaler()),
  #  ('clf', DecisionTreeClassifier())
#])

# Number of bootstrap samples and trees in the ensemble
# n_samples = len(X_train)
# n_trees = 50

#def bootstrap_sample(X, y):
#    indices = np.random.choice(len(X), size=len(X), replace=True)
#    return X[indices], y[indices]


In [123]:
# Train multiple decision trees on different bootstrap samples
from sklearn.utils import resample

n_trees = 10  # Number of trees

trees = []

for _ in range(n_trees):
    X_sample, y_sample = resample(X_train, y_train, random_state=np.random.randint(100))
    
    # Create and fit your decision tree or pipeline here
    # cloned_pipeline = ...

    trees.append(cloned_pipeline)


In [124]:
trees

[Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassif

In [125]:
import pandas as pd

# Assuming 'X_train' and 'X_test' are your feature datasets
# Assuming 'categorical_columns' is a list of column names containing categorical data

# Define the column to be one-hot encoded
column_to_encode = 'sex'

# Perform one-hot encoding on the specified column
X_train_encoded = pd.get_dummies(X_train, columns=[column_to_encode], drop_first=True)
X_test_encoded = pd.get_dummies(X_test, columns=[column_to_encode], drop_first=True)


In [126]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,Southampton,no,True


In [129]:
print("X_test_encoded shape:", X_test_encoded.shape)
print("Number of trees in the ensemble:", len(trees))


X_test_encoded shape: (268, 14)
Number of trees in the ensemble: 10


In [130]:
tree_index = 0  # Adjust the index as needed
single_tree_predictions = trees[tree_index].predict(X_test_encoded)
print("Predictions from tree", tree_index, ":", single_tree_predictions)


C:\Users\lifen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


ValueError: could not convert string to float: 'C'

In [128]:
# Aggregate predictions
predictions = np.zeros((len(X_test_encoded), n_trees))
for tree_index, tree in enumerate(trees):
    predictions[:, tree_index] = tree.predict(X_test_encoded)
    

C:\Users\lifen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


ValueError: could not convert string to float: 'C'